In [1]:
import os

In [2]:
%pwd

'/home/rohwid/Pacmann/lazada-id-reviews/notebooks'

In [3]:
# Change to the main directory
# So, it's executed from main directory
os.chdir("../")

In [4]:
%pwd

'/home/rohwid/Pacmann/lazada-id-reviews'

### Data Preprocessing Config

This code will be apply in `src/LadazaIDReviews/entity/config_entity.py`

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataDumpConfig:
    root_dir: Path
    reviews_path: Path
    input_train_path: Path
    input_test_path: Path
    output_train_path: Path
    output_test_path: Path
    params_test_size: float

@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    input_train_path: Path
    input_test_path: Path
    vectorized_train_path: Path
    vectorized_test_path: Path
    model_dir: Path
    vectorizer_model_path: Path

### Data Preprocessing Config Manager

This code will be apply in `src/LazadaIDReviews/config/configurations.py`.

What we would do?
+ Drop null values
+ Splitting the dataset to train and test data
+ Vectorize text using `TFIDF`

As stated before; let’s load, select columns, and drop null values from dataset.

In [6]:
from LazadaIDReviews.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from LazadaIDReviews.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_dump_data_config(self) -> DataDumpConfig:
        """read data dump config file and store as config entity
        then apply the dataclasses
        
        Returns:
            config: PreprocessingConfig type
        """
        data_ingest_config = self.config.ingest_from_sql
        data_dump_config = self.config.dump_data
        dataset_params = self.params

        create_directories([data_dump_config.root_dir])

        config = DataDumpConfig(
            root_dir=data_dump_config.root_dir,
            reviews_path=data_ingest_config.reviews_path,
            input_train_path=data_dump_config.input_train_path,
            input_test_path=data_dump_config.input_test_path,
            output_train_path=data_dump_config.output_train_path,
            output_test_path=data_dump_config.output_test_path,
            params_test_size=dataset_params.TEST_SIZE
        )

        return config
    
    def get_preprocessing_data_config(self) -> DataPreprocessingConfig:
        """read preprocessing config file and store as config entity
        then apply the dataclasses
        
        Returns:
            config: PreprocessingConfig type
        """
        data_dump_config = self.config.dump_data
        vectorize_config = self.config.vectorize_data
        train_config = self.config.train_model

        create_directories([vectorize_config.root_dir])

        config = DataPreprocessingConfig(
            root_dir=vectorize_config.root_dir,
            input_train_path=Path(data_dump_config.input_train_path),
            input_test_path=Path(data_dump_config.input_test_path),
            vectorized_train_path=Path(vectorize_config.vectorized_train_path),
            vectorized_test_path=Path(vectorize_config.vectorized_test_path),
            model_dir=train_config.root_dir,
            vectorizer_model_path=Path(vectorize_config.vectorizer_model_path)
        )

        return config

### Perform Preprocessing

This code in `src/LazadaIDReviews/components/preprocessing.py`.

In [8]:
import joblib
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from LazadaIDReviews import logger

class Preprocessing:
    def __init__(self, config: DataDumpConfig):
        self.config = config

    def dump_data(self) -> None:
        """dump the splited dataset to data training and testing
        """
        logger.info(f"Read reviews file.")
        dataset_reviews = pd.read_csv(self.config.reviews_path)
        dataset = dataset_reviews[["rating", "reviewContent"]].copy()
        dataset.dropna(inplace=True)
        
        logger.info(f"Split reviews file to data train and test.")
        X_train, X_test, y_train, y_test = train_test_split(
            dataset["reviewContent"], 
            dataset["rating"], 
            test_size=self.config.params_test_size
        )
        
        logger.info(f"Dump data train into {self.config.input_train_path} directory.")
        X_train.to_pickle(self.config.input_train_path)
        X_test.to_pickle(self.config.input_test_path)
        
        logger.info(f"Dump data test into {self.config.input_test_path} directory.")
        y_train.to_pickle(self.config.output_train_path)
        y_test.to_pickle(self.config.output_test_path)
        
    def vectorize_data(self) -> None:
        """vectorize the splited dataset and dump vectorizer model
        """
        vectorizer = TfidfVectorizer()
        
        logger.info(f"Load data train in {self.config.input_train_path}.")
        X_train = joblib.load(self.config.input_train_path)
        
        logger.info(f"Load data test in {self.config.input_test_path}.")
        X_test = joblib.load(self.config.input_test_path)
        
        logger.info(f"Vectorize the data.")
        X_train_vec = vectorizer.fit_transform(X_train)
        X_test_vec = vectorizer.transform(X_test)
        
        logger.info(f"Dump the vectorized data.")
        joblib.dump(X_train_vec, self.config.vectorized_train_path)
        joblib.dump(X_test_vec, self.config.vectorized_test_path)
        
        logger.info(f"Creating {self.config.model_dir} directory.")
        model_dir = str(self.config.model_dir)
        os.makedirs(model_dir, exist_ok=True)
        
        logger.info(f"Save the vectorizer model.")
        joblib.dump(vectorizer, self.config.vectorizer_model_path)

### Dump the Data Train and Data Test

This code in `src/LazadaIDReview/pipeline/step_02_preprocessing.py`.

In [9]:
try:
    config = ConfigurationManager()
    dump_data_config = config.get_dump_data_config()
    data_ingestion = Preprocessing(config=dump_data_config)
    data_ingestion.dump_data()
except Exception as e:
    logger.error(e)
    raise e

[2024-07-03 20:37:50,329: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-03 20:37:50,334: INFO: common: yaml file: metrics/params.yaml loaded successfully]
[2024-07-03 20:37:50,336: INFO: common: created directory at: artifacts]
[2024-07-03 20:37:50,339: INFO: common: created directory at: artifacts/data]
[2024-07-03 20:37:50,341: INFO: 944171181: Read reviews file.]
[2024-07-03 20:37:50,802: INFO: 944171181: Split reviews file to data train and test.]
[2024-07-03 20:37:50,811: INFO: 944171181: Dump data train into artifacts/data/X_train.pkl directory.]
[2024-07-03 20:37:50,849: INFO: 944171181: Dump data test into artifacts/data/X_test.pkl directory.]


**Debug**: Read data

In [10]:
X_train = joblib.load(dump_data_config.input_train_path)
X_train

52942     Keasyikan menikmati fitur2 dan keindahan tv yg...
180906    Barang bagus. Sesuai gambar. Dapat berfungsi d...
42906     Joss, produk recommended tinggal disambung ke ...
78698                                                  ok..
107577    Barang sesuai pesanan, respon dan pengiriman c...
                                ...                        
144839                                      Mantap betul...
140798          manstabs barang sesuai harga semoga awet...
130589                                          Recomended!
62207                                      barang sesuai...
104402    Alhamdllah mendarat dg slmat..packing rapi pen...
Name: reviewContent, Length: 21405, dtype: object

In [11]:
X_train.isnull().sum()

0

In [12]:
y_train = joblib.load(dump_data_config.output_train_path)
y_train

52942     5
180906    5
42906     5
78698     5
107577    5
         ..
144839    5
140798    5
130589    5
62207     5
104402    5
Name: rating, Length: 21405, dtype: int64

In [13]:
X_test = joblib.load(dump_data_config.input_test_path)
X_test

18124        pengiriman masuk standar range, pengemasan ok.
72091                            barang da sampai..mantab👍👍
141328                                             mantabbb
121808                          Ini berapa inci ya layarnya
185155                                               mantap
                                ...                        
10271                                        ga bisa dipake
127624       Alhamdulillah tv nya bagus dan paking nya rapi
49817     barang sudah sampai dengan selamat, tapi tidak...
501       Produk sesuai dengan deskripsi, sampai saat in...
58544                                 BARANG SESUAI PESANAN
Name: reviewContent, Length: 85624, dtype: object

In [14]:
X_test.isnull().sum()

0

In [15]:
y_test = joblib.load(dump_data_config.output_test_path)
y_test

18124     4
72091     5
141328    5
121808    3
185155    5
         ..
10271     1
127624    5
49817     5
501       5
58544     4
Name: rating, Length: 85624, dtype: int64

### Vectorize the Data Train and Data Test

This code in `src/LazadaIDReview/pipeline/step_02_preprocessing.py`.

In [16]:
try:
    config = ConfigurationManager()
    preprocessing_config = config.get_preprocessing_data_config()
    preprocessing = Preprocessing(config=preprocessing_config)
    preprocessing.vectorize_data()
except Exception as e:
    logger.error(e)
    raise e

[2024-07-03 20:40:04,993: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-03 20:40:04,997: INFO: common: yaml file: metrics/params.yaml loaded successfully]
[2024-07-03 20:40:04,998: INFO: common: created directory at: artifacts]
[2024-07-03 20:40:05,000: INFO: common: created directory at: artifacts/preprocessing]
[2024-07-03 20:40:05,001: INFO: 944171181: Load data train in artifacts/data/X_train.pkl.]
[2024-07-03 20:40:05,032: INFO: 944171181: Load data test in artifacts/data/X_test.pkl.]
[2024-07-03 20:40:05,154: INFO: 944171181: Vectorize the data.]
[2024-07-03 20:40:06,077: INFO: 944171181: Dump the vectorized data.]
[2024-07-03 20:40:06,096: INFO: 944171181: Creating artifacts/models directory.]
[2024-07-03 20:40:06,098: INFO: 944171181: Save the vectorizer model.]


**Debug**: Read data

In [17]:
X_train_vec = joblib.load(preprocessing_config.vectorized_train_path)
X_train_vec

<21405x13745 sparse matrix of type '<class 'numpy.float64'>'
	with 251629 stored elements in Compressed Sparse Row format>

In [18]:
X_test_vec = joblib.load(preprocessing_config.vectorized_test_path)
X_test_vec

<85624x13745 sparse matrix of type '<class 'numpy.float64'>'
	with 984786 stored elements in Compressed Sparse Row format>